In [1]:
%load_ext autoreload
%autoreload 2
import glob
import geopandas as gpd

directory = '/scratch/6256481/'

import sys
sys.path.insert(0, directory + f'python/py_code/')

from support import open_mg_file
from get_orthogonals import get_orthogonals
from dem import find_dem
from cycle_identification import find_connected_side, get_cycles

from support import create_dir, open_all_files, get_local_utm_projection
import glob

import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
c = 'na'
i = 74
code_failure = ''

    # try:
        
projection       = 'EPSG:3857'
projection_coord = 'EPSG:4326'


vector_file = glob.glob(directory + f'input/SWOT_vector/{c}*{i}_*shp')[0]
node_file   = glob.glob(directory + f'input/SWOT_nodes/{c}*{i}_*shp')[0]

df_vector       = gpd.read_file(vector_file)
if df_vector.crs is None:
    df_vector = df_vector.set_crs(projection_coord)

df_vector = df_vector.to_crs(projection)
df_vector_whole = df_vector.copy()

# add local projection to dataframe
df_vector_whole = get_local_utm_projection(df_vector_whole)

df_vector_whole = df_vector_whole[df_vector_whole['type'] != 6]      # exclude ghostreaches
df_vector_whole = df_vector_whole[df_vector_whole.reach_len > (500)] # minimum reachLength



In [3]:

# finds connected reaches and creates cycles
cycleFile = directory + f'results/cycles/{c}_{i}_cycle.shp'
df_cycle, dfConnectionNodes  = find_connected_side(df_vector_whole, projection, cycleFile)

enter get_cyclessss


In [4]:
dfConnectionNodes

,node_id,reach_id,geometry,distance,side
0,0,74100300015,POINT (-9971888.647 3529784.370),0,Start
1,0,74100400015,POINT (-9971888.647 3529784.370),0,Start
2,0,74100400025,POINT (-9971888.647 3529784.370),0,Start
0,1,74100400015,POINT (-9979656.312 3534255.223),7744.234055,Mid
1,1,74100400495,POINT (-9979656.312 3534255.223),7744.234055,Mid
...,...,...,...,...,...
1,4638,74300500061,POINT (-10259267.194 3498954.973),7109.311977,End
0,4639,74300500061,POINT (-10250962.273 3504834.011),8802.975277,End
1,4639,74300500071,POINT (-10250962.273 3504834.011),8802.975277,End
0,4640,74300500071,POINT (-10248603.741 3522339.095),15217.608823,End


In [5]:
dfConnectionNodes[dfConnectionNodes.reach_id == 74294500151]

,node_id,reach_id,geometry,distance,side
2,3592,74294500151,POINT (-10982299.782 4971802.411),2371.402118,End
2,3593,74294500151,POINT (-10991214.577 4970574.999),6826.851433,End


In [8]:
dfConnectionNodes[dfConnectionNodes.node_id.isin([3592, 3593])]

,node_id,reach_id,geometry,distance,side
0,3592,74294500111,POINT (-10982299.782 4971802.411),2371.402118,End
1,3592,74294500121,POINT (-10982299.782 4971802.411),2371.402118,End
2,3592,74294500151,POINT (-10982299.782 4971802.411),2371.402118,End
0,3593,74294500121,POINT (-10991214.577 4970574.999),6826.851433,End
1,3593,74294500161,POINT (-10991214.577 4970574.999),6826.851433,End
2,3593,74294500151,POINT (-10991214.577 4970574.999),6826.851433,End


In [14]:
dfTest = dfConnectionNodes.copy()
# dfTest = dfTest[dfTest.node_id.isin([3592, 3593])]
grouped = dfTest.groupby('reach_id')

edges_2 = [tuple(group_df.node_id.tolist()) for group, group_df in grouped if 
           (len(group_df.node_id.tolist()) > 1) &
           (len(group_df.node_id.tolist()) < 3)]

In [7]:
df_cycle[df_cycle.reach_id == 74294500151]

,reach_id,Key,Value,Start,End,Mid,Mid1,Mid2,cycleID,cycle
3922,74294500151,74294500151.0,"[74294500161, 74294500121, 74294500111]","[74294500121, 74294500111]","[74294500161, 74294500121]",nan,nan,nan,[28],True


In [32]:
D = df_cycle.copy()
A = get_cycles(df_vector_whole, dfConnectionNodes)


enter get_cyclessss
[Index([74300400125, 74300400475, 74300400495], dtype='object', name='reach_id'), Index([74300400125, 74300400495, 74300400135, 74300400515, 74300400505], dtype='object', name='reach_id'), Index([74300400135, 74300400515, 74300400525], dtype='object', name='reach_id'), Index([74230100051, 74230100021], dtype='object', name='reach_id'), Index([74230700081, 74230700031, 74230700041, 74230700101, 74230700091,
       74230700071],
      dtype='object', name='reach_id'), Index([74270900051, 74270900071, 74270900081], dtype='object', name='reach_id'), Index([74294500091, 74294500321, 74294500101, 74294500331, 74294500341], dtype='object', name='reach_id'), Index([74294500171, 74294500351, 74294500361, 74294500181], dtype='object', name='reach_id'), Index([74249200363, 74249200311, 74249200303, 74249200371], dtype='object', name='reach_id'), Index([74244100043, 74244100053, 74244100231, 74244100103, 74244100223], dtype='object', name='reach_id'), Index([74230100021, 742301

In [43]:
res = list(set([ele for ele in edges_2 
            if edges_2.count(ele) > 1])) 

A = [[1,2], [3,4]]
for i, I in enumerate(res):
    rows = dfConnectionNodes[dfConnectionNodes.node_id.isin(I)]
    singleCycle = rows[rows.reach_id.duplicated()].reach_id.values
    A.append(list(singleCycle))
A

[[1, 2],
 [3, 4],
 [74300300075, 74300500111],
 [74222400141, 74222400161],
 [74300400155, 74300400165],
 [74294500121, 74294500151]]